In [3]:
import os
import torch
from pylab import *
import numpy as np
from numpy import zeros
from tqdm import tqdm
import torch.nn as nn
from model import UNet
from scipy import ndimage
import SimpleITK as sitk
from numpy import ndarray
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import transforms
from result_submission import SegThorSubmission, Rescale, ToTensor, Normalize

In [4]:
def tensor_to_numpy(tensor):
    t_numpy = tensor.cpu().numpy()
    t_numpy = np.transpose(t_numpy, [0, 2, 3, 1])
    t_numpy = np.squeeze(t_numpy)

    return t_numpy

In [19]:
def test():
    test_path = 'data/test'
    for patient in tqdm(os.listdir(test_path)): 
        count = 0

        file = patient
        x = file.split(".")
        filename = x[0] + '.' + x[1]
        test_set = SegThorSubmission(test_path, patient=patient, phase='test',
                                   transform=transforms.Compose([
                                       Rescale(0.25),
                                       Normalize(),                           
                                       ToTensor()
                                   ]))
        
        test_loader = torch.utils.data.DataLoader(dataset=test_set, 
                                                  batch_size=11, 
                                                  shuffle=False)

        print(len(test_set))
        mask_i = zeros([len(test_set),  128, 128])
       # a = zeros([3,5])
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        model = torch.load("models/model.pt")
        model.eval()
        model.to(device)
        
        
        with torch.no_grad():
            for batch_idx, image in enumerate(test_loader):     
                images = image.to(device, dtype=torch.float)        
                outputs = model(images)

                images = tensor_to_numpy(images)
                outputs = tensor_to_numpy(outputs)                
           #     mask_i = np.zeros((len(test_set), images.shape[1], images.shape[2]))
            #    print(outputs.shape)
                for i in range(outputs.shape[0]):
                    for j in range(outputs.shape[3]):
                        for x in range(outputs.shape[1]):
                            for y in range(outputs.shape[2]):
                                if outputs[i, x, y, j] > 0.5:               ## For Eusophagus:  Red
                                    mask_i[count, x, y] = j
                    count = count + 1

        volume_numpy = zeros([len(test_set),  512, 512])
        for s in range(0,mask_i.shape[0]):
            slice_v = mask_i[s,:,:]           
            slice_v = slice_v.astype(float32)
            slice_v = ndimage.interpolation.zoom(slice_v, zoom=4, order=0, mode='nearest', prefilter=True)
            volume_numpy[s, :, :] = slice_v
        
        print(volume_numpy.shape)
        volume = sitk.GetImageFromArray(volume_numpy, isVector=False)
        print(volume.GetSize())
        sitk.WriteImage(sitk.Cast( volume, sitk.sitkUInt8 ), filename, True) 

In [21]:
if __name__ == "__main__":
    test()

  0%|          | 0/1 [00:00<?, ?it/s]

data/test/Patient_48.nii.gz
shape of the volume while reading  (205, 512, 512)
Length after assinging to list 205
205
(205, 512, 512)
(512, 512, 205)


100%|██████████| 1/1 [00:22<00:00, 22.47s/it]
